<a href="https://colab.research.google.com/github/sanjeevanahilan/nanoChatGPT/blob/master/chatgpt_dev_teaching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's train models for generating Shakespeare with different sentiments.

Happy Shakespeare will say things like: 

*“Nay, thanks, then, I do meet change, this Romeo. 
The pleasure of his hair!”*

Sad Shakespear will say things like:


*“The senators's dead, of their world:
Be not for your friends.”*



In [1]:
# Notebook created by Sanjeevan Ahilan
# GPT implementation was borrowed from Andrej Karpathy 
# https://colab.research.google.com/drive/1JMLa53HDuA-i7ZBmqV7ZnA3c_fvtXnx-?usp=sharing
# My repo: https://github.com/sanjeevanahilan/nanoChatGPT
# My twitter: https://twitter.com/sanjeevanahilan

import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)


In [2]:
# Download Shakespeare
%time
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 7.15 µs
--2023-03-18 11:49:44--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.03s   

2023-03-18 11:49:45 (40.0 MB/s) - ‘input.txt’ saved [1115394/1115394]



Rather than use character level tokenizer lets use tiktoken; Openai's implementation of a Byte Pair Encoding (BPE) tokenizer

In [3]:
!pip install tiktoken
import tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.8 MB/s eta 0:00:00


In [4]:
enc = tiktoken.get_encoding("gpt2")
vocab_size = 50257

In [5]:
# You can see that the sequence of characters 'the' has been encoded as a single
# number because it is commonly reoccuring whereas 'thh' requires two numbers to
# encode 'th' and 'h'

print(enc.encode('the'))
print(enc.encode('thh'))
print(enc.decode([400]))

[1169]
[400, 71]
th


In [6]:
# Train and test splits
data = torch.tensor(enc.encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPT(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = GPT()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

6.684497 M parameters
step 0: train loss 10.9982, val loss 11.0105
step 100: train loss 6.3955, val loss 6.4657
step 200: train loss 6.0255, val loss 6.1584
step 300: train loss 5.7608, val loss 5.9578
step 400: train loss 5.4809, val loss 5.7327
step 500: train loss 5.2543, val loss 5.5400
step 600: train loss 5.1017, val loss 5.4008
step 700: train loss 4.9718, val loss 5.3382
step 800: train loss 4.8850, val loss 5.2539
step 900: train loss 4.8084, val loss 5.1674
step 1000: train loss 4.7187, val loss 5.0770
step 1100: train loss 4.6621, val loss 5.1166
step 1200: train loss 4.5916, val loss 5.0159
step 1300: train loss 4.5599, val loss 5.0297
step 1400: train loss 4.4998, val loss 4.9929
step 1500: train loss 4.4481, val loss 4.9933
step 1600: train loss 4.4182, val loss 4.9204
step 1700: train loss 4.4202, val loss 4.9445
step 1800: train loss 4.3644, val loss 4.9217
step 1900: train loss 4.3479, val loss 4.9221
step 2000: train loss 4.2925, val loss 4.8603
step 2100: train loss 

In [7]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(enc.decode(m.generate(context, max_new_tokens=100)[0].tolist()))

! what any consul
matter they have we be for so dishonour
Redpt'd in our courtesy,
Or his highness i' the embracements, nor feeds had an err:
I have the hallissign daughter to pawn'd.
and stout see me: but God they forget
And meet the tribranch man,
Of a brace art'd; I'll plant show't
From whom I a joyful with a chair access with callat.

HENRY


In [8]:
!pip install transformers
from transformers import pipeline
sentiment_pipeline = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 90.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 93.8 MB/s eta 0:00:00


emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [9]:
inps = ["I'm the king of the world!", 
        "I'll be back.",
        "The cake is a lie", 
        "To be forgotten is worse than death",
        "All happy families are alike; each unhappy family is unhappy in its own way.",
        "You don't need a reason to help people",
        ]
res = sentiment_pipeline(inps)

for i in range(len(inps)):
  res[i]['text'] = inps[i]
  print(res[i])

{'label': 'POS', 'score': 0.9771729707717896, 'text': "I'm the king of the world!"}
{'label': 'POS', 'score': 0.5481611490249634, 'text': "I'll be back."}
{'label': 'NEG', 'score': 0.7581188678741455, 'text': 'The cake is a lie'}
{'label': 'NEG', 'score': 0.8209365606307983, 'text': 'To be forgotten is worse than death'}
{'label': 'NEU', 'score': 0.7874237895011902, 'text': 'All happy families are alike; each unhappy family is unhappy in its own way.'}
{'label': 'NEU', 'score': 0.8731082081794739, 'text': "You don't need a reason to help people"}


In [10]:
def get_reward(text, mode):
  sent = sentiment_pipeline(text)
  if mode == '+ve':
    labels = torch.tensor([a['label']=='POS' for a in sent],dtype=torch.float16).unsqueeze(-1).to(device)
  elif mode == '-ve':
    labels = torch.tensor([a['label']=='NEG' for a in sent],dtype=torch.float16).unsqueeze(-1).to(device)
  else:
    raise ValueError('Unknown Mode')
  
  weights = torch.tensor([a['score'] for a in sent],dtype=torch.float32).unsqueeze(-1).to(device)
  
  rewards = labels * weights # (B, 1)

  return rewards

In [11]:
def flatten(l):
    return [item for sublist in l for item in sublist]
print('Rewards in +ve mode')
list(zip(inps, flatten(get_reward(inps, '+ve').tolist())))


Rewards in +ve mode


[("I'm the king of the world!", 0.9771729707717896),
 ("I'll be back.", 0.5481611490249634),
 ('The cake is a lie', 0.0),
 ('To be forgotten is worse than death', 0.0),
 ('All happy families are alike; each unhappy family is unhappy in its own way.',
  0.0),
 ("You don't need a reason to help people", 0.0)]

In [12]:
print('Rewards in -ve mode')
list(zip(inps, flatten(get_reward(inps, '-ve').tolist())))

Rewards in -ve mode


[("I'm the king of the world!", 0.0),
 ("I'll be back.", 0.0),
 ('The cake is a lie', 0.7581188678741455),
 ('To be forgotten is worse than death', 0.8209365606307983),
 ('All happy families are alike; each unhappy family is unhappy in its own way.',
  0.0),
 ("You don't need a reason to help people", 0.0)]

In [13]:
eval_interval_rlhf = 20
max_iters_rlhf = 1000


In [14]:
from torch.distributions import Categorical
class RLHF(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, idx, targets=None):
        return self.model(idx, targets)
     
    def generate(self, idx, max_new_tokens, block_size, ref_model=None):
        # idx is (B, T) array of indices in the current context
        log_probs = torch.tensor([]).to(device)
        log_probs_ref = torch.tensor([]).to(device)
        
        for i in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]

            # get the predictions
            logits, loss = self(idx_cond)

            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            
            # logits define instance of Iategorical class
            m = Categorical(logits=logits)
            
            # sample from the distribution
            idx_next = m.sample()
            
            # get the log probability and append to running sequence
            log_probs_idx_next = m.log_prob(idx_next)    
            log_probs = torch.cat((log_probs, log_probs_idx_next.view(-1,1)), dim=1)
            
            if ref_model is not None:
              # get log probability of sample idx_next under the reference model
              logits_ref, _ = ref_model(idx_cond)
              logits_ref = logits_ref[:, -1, :] # becomes (B, C)
            
              m_ref = Categorical(logits=logits_ref)
              log_probs_ref_idx_next = m_ref.log_prob(idx_next)    
              log_probs_ref = torch.cat((log_probs_ref, log_probs_ref_idx_next.view(-1,1)), dim=1)

            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next.view(-1,1)), dim=1) # (B, T+1)

        return idx, log_probs, log_probs_ref

In [15]:
import copy
ref_model = copy.deepcopy(model)

In [16]:
import time
import numpy as np

RLHFmodel = RLHF(model)
RLHFmodel.to(device)

ref_model.to(device)

actor_optimizer = torch.optim.AdamW(RLHFmodel.parameters(), lr=1e-3)
X, Y = get_batch('train') # fetch the very first batch
X = torch.ones((X.shape[0], 1), dtype=torch.long).to(device) # for now there is no prompt
X = X*enc.encode('The')[0] # start with ''The'
t0  = time.time()
max_new_tokens = block_size
rews_all = []
actor_loss_all = []
mode = '+ve'
ref_coef = 0.2
e_coef = 0.1
for iter in range(max_iters_rlhf):

  states, log_probs, log_probs_ref = RLHFmodel.generate(
      X, max_new_tokens, block_size, ref_model=ref_model)

  states = states[:,-max_new_tokens:]
  log_probs = log_probs[:,-max_new_tokens:] # (B, max_new_tokens)
  if ref_model is not None:
    log_probs_ref = log_probs_ref[:,-max_new_tokens:] # (B, max_new_tokens)
  
  rewards = get_reward([enc.decode(s.tolist()) for s in states], mode)
  
  pg = (rewards+ref_coef*log_probs_ref-e_coef*log_probs)* log_probs.squeeze()
  
  # log(1) = 0
  # -log(1/N) = log(N)

  # when ref_coef=e_coef this is equivalent to penalising for KL divergence
  # pg = (rewards-ref_coef*(log_probs-log_probs_ref)* log_probs.squeeze() 
  
  actor_loss = -pg.sum()

  actor_optimizer.zero_grad(set_to_none=True)
  actor_loss.backward()
  actor_optimizer.step()

  rews_all.append(rewards.mean().detach().cpu().numpy())
  actor_loss_all.append(actor_loss.detach().cpu().numpy())

  if iter % eval_interval_rlhf == 0:
      t1 = time.time()
      print('\n')
      print(f'iter: {iter}, time: {t1-t0}')
      print(f'Actor loss: {np.mean(actor_loss_all[-eval_interval_rlhf:])}')
      print(f'rets: {np.mean(rews_all[-eval_interval_rlhf:])}')

      textRLHF = RLHFmodel.generate(X, 2*max_new_tokens, block_size, ref_model=None)[0]
      for i in range(1):
          text_i = textRLHF[i,:]
          print(enc.decode(text_i.tolist()))




iter: 0, time: 2.0925092697143555
Actor loss: -1174.6953125
rets: 0.15557555854320526
The nobles
withoveseed
Of our dimina to the narrow, of mine,
For this earth brawling some fathers, thou shalt
Con exceeds bow an ostows that were I
actioning Roe that word;
For what 'twas, likeers Lartft,
And whose secret maid


iter: 20, time: 43.471943855285645
Actor loss: -1276.6146240234375
rets: 0.08284272998571396
The deceived!

SICINs:
The very faults of Norfolk is the cousin of sword;
And, by me how so, seldom the last of that wounds
Or much power home his clish'd utterhire, wine of that dame'd
Of my life as the loss of death, be


iter: 40, time: 83.07254672050476
Actor loss: -1171.9931640625
rets: 0.12373824417591095
The bride, wives are
Mis kneel to repent him for heaven and in debt.
Look, thou fly, makes your beauty is his rash, I will not
Say a side that taught their robbery.
Mis up the demands Hastings.

JULNot SAL:
Your brother is drawn, and in


iter: 60, time: 123.09890699386597
Act

In [17]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(enc.decode(m.generate(context, max_new_tokens=200)[0].tolist()))

!
You be right,
And triumph; am so, my good
to of general; if thou husband
As never sunder best against my blood.

DUKE VINCENTIO:
Then like a Romansers, art speed at the queen,
Fromagenetings, my tears. But

ulet:
Which was what I break scarce; where then.
For this we'll bear the dewance of man!

POLIXENES:
I'll swear soon I will say out a happyker you.
Hear thee.

DUKE VINCENTIO:
But since our life, Lord, Titus me,
For I love this wed is the battle your man.

CATESBY:
Farewell you, why.

CLAUDIO:
 if you hear that live Not gone, sent, let our headsarer
wixt the XIant cannot talkings
Should is cl


1. We trained a GPT model to reproduce Shakespeare
2. We built a reward model by repurposing a Huggingface sentiment classifier
3. We fine tuned the GPT model using reinforcement learning. 
4. The model over-optimised the reward so we penalised it for moving too far from a reference model
5. We found it to be far too repetitive and so we added in an entropy bonus to encourage diverse outputs

---





